### Carga diaria de Analisis de Sentimientos

*  Cargar los datos
 * Cargar TRM Historica y calcular deltas
*  Calcular correlaciones spearman y pearson
 * Correlaciones con diferentes Lags hasta 15 días
* Validar otras variables

In [1]:
import pandas as pd
import tweepy
from datetime import *
import datetime as dt
import json, re
import feedparser
from time import mktime

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import mysql.connector
connection = mysql.connector.connect(host='localhost', port=3307,
                                         database='tesis',
                                         user='root',
                                         password='Cidifo03%%')

In [5]:
# Analisis de Sentimientos
##librerias ingles
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

#librerías Español
import joblib
from  classifier import SentimentClassifier
clf = SentimentClassifier()

# importaciones Tokenizacion

import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
pst = PorterStemmer()
import string
from nltk.probability import FreqDist
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
stopWEnglish = set(stopwords.words('english'))
stopWSP = set(stopwords.words('spanish'))

from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\ProgramData\Anaconda3\envs\T\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CristhianJohnatanIzq\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stop

In [6]:
pd.set_option("display.max_colwidth", 100)

In [7]:
#fechaHoy=datetime.datetime.now().date()
fechaHoy=dt.datetime.now().date()

#  Funciones

In [8]:
def limpiar_terminos(texto):    
    texto=" ".join(filter(lambda x:x[0]!='#', texto.split()))
    texto=" ".join(filter(lambda x:x[0]!='@', texto.split()))
    texto=" ".join(filter(lambda x:x[0:4]!='http', texto.split()))
    return texto

In [9]:
def limpiar_texto(texto):
    """
    Función para realizar la limpieza de un texto dado.
    """    
    texto=limpiar_terminos(texto)
    
    # Eliminamos los caracteres especiales
    texto = re.sub(r'\W', ' ', str(texto))
    texto = re.sub(r'\d', ' ', str(texto))
    # Eliminamos comas y /
    texto = re.sub(r',', ' ', str(texto))
    texto = re.sub(r'/', ' ', str(texto))
    # Eliminado las palabras que tengo un solo caracter
    texto = re.sub(r'\s+[a-zA-Z]\s+', ' ', texto)
    # Sustituir los espacios en blanco en uno solo
    texto = re.sub(r'\s+', ' ', texto, flags=re.I)
    # remover \n
    #texto = re.sub(r'\n', '', texto)
    texto = texto.replace('\n', '')
    texto = texto.replace('º', '')    
      
    # Convertimos textos a minusculas
    texto = texto.lower()
    return texto

In [10]:
def analisisSentimientosIngles(text):
    resultado = sid.polarity_scores(text)
    return resultado["compound"]

## enEspañol
def analisisSentimientosEspanol(text):
    resultado= clf.predict(text)
    resultado= resultado*2-1
    #resultado= resultado
    return resultado


def seleccionAnalisis(fila):
    #print(fila)
    if fila["lang"] == "en" :
        return analisisSentimientosIngles(str(fila["full_text"]))
    elif fila["lang"] == "es" :
        return analisisSentimientosEspanol(str(fila["full_text"]))

#tokenizacion
def tokenizcionIngles(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"''"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    textMin = word_tokenize(textPunt.lower())
    textSW = [x for x in textMin if x not in stopWEnglish]
    tokenStm = [None] * len(textSW)
    for i in range(0, len(textSW)):
        tokenStm[i]=pst.stem(textSW[i])
    return tokenStm
    #fdist = FreqDist(tokenStm)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def tokenizcionEspanol(text):
    textPunt = text.translate(str.maketrans('','',string.punctuation))
    textPunt = textPunt.translate(str.maketrans('','','1234567890'))
    textPunt = textPunt.translate(str.maketrans('','','’'))
    textPunt = textPunt.translate(str.maketrans('','',"'"))
    textPunt = textPunt.translate(str.maketrans('','','✅'))
    #quitarHasTags
    
    ##textMin = filter(lambda x:x[0]!='#', word_tokenize(textPunt.lower()))
    textMin = word_tokenize(textPunt.lower())    
    
    textSW = [x for x in textMin if x not in stopWSP]
    textSW = [x for x in textSW if x not in word_rem['Palabras'].values] 
    stemmed_text = [stemmer.stem(i) for i in textSW]
    return stemmed_text
    #fdist = FreqDist(stemmed_text)
    #return pd.DataFrame(fdist.items(), columns=['word', 'frec'])

def seleccionTokenizacion(fila,var):
    if fila["lang"] == "en" :
        return tokenizcionIngles(str(fila[var]))
    elif fila["lang"] == "es" :
        return tokenizcionEspanol(str(fila[var]))


#RSS definicion de métodos
def parse(url):
    return feedparser.parse(url)

def get_source(parsed):
    feed= parsed['feed']
    return {
        'link': feed['link'],
        'title': feed['title'],
        'subtitle': feed['subtitle']
    }

def get_articles(parsed):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            articles.append({
                'id': entry['id'],
                'link': entry['link'],
                'title': entry['title'],
                'summary':entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def get_articles_resultado(parsed,fuenteRSS):
        articles = []
        entries= parsed['entries']
        for entry in entries:
            #partDate =entry['published_parsed'].replace('(','')replace(')','').split(",")
            #almacenarFuente            
            # validacion si falta una dato            
            if 'summary' not in entry :
                entry['summary']=""
            if 'title' not in entry :
                entry['title']=""    
            articles.append({
                'fuenteRss':fuenteRSS,
                'idP': entry['id'],                
                'texto': entry['title']+". "+entry['summary'],
                'published': entry['published_parsed']                
            })
        return articles
    
def getFechaFromRss(fechaStruct):
    #print (fechaStruct)
    if fechaStruct is None:
        #print ("entro")
        return datetime.now()
    else:
        return datetime.fromtimestamp(mktime(fechaStruct))

In [11]:
### Funciones para modelo de tópicos
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts,bigram_mod,trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def impresionTopicos(mod,nPalabras,nTopicos):
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    for topico in topicos:
        print(topico)    
        
def impresionTopicosLimp(mod,nPalabras,nTopicos):
    #Impresion limpia de Tópicos
    topicos = mod.print_topics(num_words=nPalabras, num_topics=nTopicos)
    i=0
    for topico in topicos:
        i=i+1
        textoAux = re.sub(r'\W', ' ', str(topico))
        textoAux = re.sub(r'\s+[a-zA-Z]\s+', ' ', textoAux)
        textoAux = re.sub(r'[0-9]', ' ', textoAux)
        textoAux = re.sub(r'\s+', ' ', textoAux, flags=re.I)
        print(str(i),textoAux)

## Carga de los datos

In [12]:
### Twitter
sql_select_Query = "select * from tesis.t_base_twitter"
base_twitter=pd.read_sql(sql_select_Query, connection)

In [13]:
#Rss
sql_select_Query = "select * from tesis.t_base_rss"
base_rss=pd.read_sql(sql_select_Query, connection)

### TWITER

In [14]:
base_twitter

,index,screen_name,id,id_str,name,location,Idioma,concepto,impacto,NoTweets
0,0,sergiodigra,NaN,NaN,None,None,es,None,NaN,100
1,1,Victor_Malagon,NaN,NaN,None,None,es,None,NaN,100
2,2,jscorrales,NaN,NaN,None,None,es,None,NaN,100
3,3,_minecogob,NaN,NaN,None,None,es,None,NaN,100
4,4,elEconomistaes,NaN,NaN,None,None,es,None,NaN,100
5,5,FT,NaN,NaN,None,None,en,None,NaN,100
6,6,rodriguezbraun,NaN,NaN,None,None,es,None,NaN,100
7,7,josecdiez,NaN,NaN,None,None,es,None,NaN,100
8,8,suanzes,NaN,NaN,None,None,es,None,NaN,100
9,9,wef,NaN,NaN,None,None,es,None,NaN,100


In [15]:
###TWEETER
# autenticación
# keys and tokens from the Twitter Dev Console 
consumer_key = '5BOcNnQsuIxDTBejlZ75xBqZ0'
consumer_secret = 'c6Cnx7SItN5Gda6K8hgtPo3tPyI18TU9eUh28jx3d4lDceewiG'
access_token = '927023579727884288-1jTaXF5wmUbX2NeYzUcSlmzXQMca4kU'
access_token_secret = 'Y62wUwgoWjfHGw5S0jlYH7w8ulmlRzaFcxbGjBP0wLin9'

#creación del objeto para extraer información
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
#obtejo
api= tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [16]:
#Cargar Tweets
tweets = []
fechaDelTweetAnterior=fechaHoy
    
for index, row in base_twitter.iterrows():
    #for amigo in df:
    i=0
    print(str(i)+" "+row['screen_name'])
    tweetsAmigo=tweepy.Cursor(api.user_timeline, screen_name=row['screen_name'], tweet_mode="extended").items(row['NoTweets'])
    for tweet in tweetsAmigo:
        tweets.append({
            'id':tweet.id,
            'full_text':tweet.full_text,
            'created_at':tweet.created_at,
            #'lang':row['Idioma'],
            'lang':tweet.lang,
            'id_fuente':row['screen_name']
        })
    i=i+1 

0 sergiodigra
0 Victor_Malagon
0 jscorrales
0 _minecogob
0 elEconomistaes
0 FT
0 rodriguezbraun
0 josecdiez
0 suanzes
0 wef
0 ecofeminita
0 epeconomia
0 RevistaDinero
0 bbcmundo
0 BluRadioCo
0 elespectador
0 RevistaSemana
0 ELTIEMPO
0 elcolombiano
0 lafm
0 NoticiasRCN
0 rcnradio
0 MailOnline
0 clarincom
0 NBCNews
0 rcnmundo
0 CaracolRadio
0 NoticiasCaracol
0 NTN24
0 NASA
0 PublimetroCol
0 Adcamach
0 Adriana_Guzman
0 agaviriau
0 AlbertoBernalLE
0 alhosu
0 amaldon19
0 Anamibanez
0 BancoRepublica
0 BvcColombia
0 CamiloDeGuzman
0 carlos_g_ospino
0 castellanosgd
0 CehernandezEcon
0 Cesaretamayo
0 estebanpie
0 Fedesarrollo
0 FocoEconomico
0 Fondosdepension
0 freddykastro
0 IECarballo
0 jagallegod
0 japebu
0 JavierMejiaC
0 jleibovi
0 Jllanosala
0 JoMalagon
0 JorgeARestrepo
0 JorgeATovar
0 JoseILOPEZ
0 jrobertoacosta1
0 juanf_vargas
0 juansrobledo
0 JusTributaria
0 LeopoldoTweets
0 LuisFelipeHenao
0 LuisFerMejia
0 mahofste
0 MantillaIgnacio
0 mariovalencia01
0 MauricioCard
0 Maurotov
0 Melende

In [17]:
tweetsDF=pd.DataFrame(tweets)
pd.set_option('display.max_colwidth', -1)
tweetsDFProcesado=tweetsDF.copy()
### limpiar texto
tweetsDFProcesado["full_text_Limpio"]=tweetsDFProcesado.full_text.apply(limpiar_texto)

In [18]:
tweetsDFProcesado.lang.value_counts()

es     7238
en     1309
und    179 
ca     22  
pt     19  
it     10  
fr     8   
in     3   
no     2   
fi     2   
hu     2   
ro     1   
tl     1   
de     1   
eu     1   
nl     1   
cs     1   
Name: lang, dtype: int64

In [19]:
tweetsDFProcesado['SentSQDM']=tweetsDFProcesado.full_text_Limpio.apply(sentiment.sentiment)*2-1

In [20]:
#tweetsDFProcesado['SentTesis']=tweetsDFProcesado.full_text_Limpio.apply(seleccionAnalisis)
tweetsDFProcesado['SentTesis']=tweetsDFProcesado.apply(lambda row: seleccionAnalisis(row), axis=1)

In [21]:
tweetsDFProcesado['SentSQDM']=tweetsDFProcesado['SentSQDM'].round(3)
tweetsDFProcesado['SentTesis']=tweetsDFProcesado['SentSQDM'].round(3)

In [22]:
tweetsDFProcesado.head(10)

,id,full_text,created_at,lang,id_fuente,full_text_Limpio,SentSQDM,SentTesis
0,1487580108370202629,RT @paolave24: ‘She immediately saw herself’: how Encanto strikes a major chord in a diverse world https://t.co/CeZHf36Vjy,2022-01-30 00:15:10,en,sergiodigra,rt she immediately saw herself how encanto strikes major chord in diverse world,0.66,0.66
1,1487204810227134468,"RT @unwomenchief: Today’s MoU, signed between @AgendaCAF &amp; @UN_Women, marks a historic step in our joint efforts to ensure and strengthen w…",2022-01-28 23:23:52,en,sergiodigra,rt today mou signed between amp marks historic step in our joint efforts to ensure and strengthen,0.10,0.10
2,1486789436415397892,"RT @PiliCancela: 🤝Nuevo acuerdo entre la @CooperacionESP, @_minecogob y la @AgendaCAF para sentar las bases de futuras iniciativas, con el…",2022-01-27 19:53:19,es,sergiodigra,rt nuevo acuerdo entre la la para sentar las bases de futuras iniciativas con el,-0.74,-0.74
3,1486567827712749572,"RT @omarperotti: #SantaFe+Conectada: Inclusión Digital y Transformación Educativa.\nJunto a @alferdez, @Rojasarroyo y Gustavo Béliz, firmamo…",2022-01-27 05:12:43,es,sergiodigra,rt inclusión digital transformación educativa junto y gustavo béliz firmamo,0.08,0.08
4,1485612341442101257,"RT @La_network: El río Rímac, los residentes en sus zonas aledañas y el público en general, se beneficiarán con la firma de Cooperación Téc…",2022-01-24 13:55:58,es,sergiodigra,rt el río rímac los residentes en sus zonas aledañas el público en general se beneficiarán con la firma de cooperación téc,-0.47,-0.47
5,1484963958322827273,RT @Ernstr_97: Our new book is out! https://t.co/EdHQ4hp38s,2022-01-22 18:59:31,en,sergiodigra,rt our new book is out,0.11,0.11
6,1484894273355108355,RT @carlos_santiso: Felicitaciones a Colombia por estos avances y logros en su política de #InteligenciaArtificial. Felices de apoyarles de…,2022-01-22 14:22:37,es,sergiodigra,rt felicitaciones colombia por estos avances logros en su política de felices de apoyarles de,-0.93,-0.93
7,1484894203599626243,RT @AgendaCAF: América Latina necesita modernizar sus arreglos institucionales para que la agenda anticorrupción se sintonice con la aceler…,2022-01-22 14:22:20,es,sergiodigra,rt américa latina necesita modernizar sus arreglos institucionales para que la agenda anticorrupción se sintonice con la aceler,-1.00,-1.00
8,1484892191277334532,@JuanPSalazarS @ClaudiaLopez @AgendaCAF Gracias JuanPa por liderar esta causa!,2022-01-22 14:14:21,es,sergiodigra,gracias juanpa por liderar esta causa,-0.46,-0.46
9,1484837974416277505,RT @mickdonaldson: Felicidades @AgendaCAF y @onuhabitat por este oportuno debate sobre derechos digitales. Gran conducción de @porsiempreli…,2022-01-22 10:38:54,es,sergiodigra,rt felicidades por este oportuno debate sobre derechos digitales gran conducción de,-0.33,-0.33


# RSS

In [23]:
base_rss[:8]

,index,id,FuentesRSS,lenguaje,Fuente,Pais,tema,Concepto,funciona,concepto_estandar,impacto,inter_local,DescxErrores
0,0,1.00,http://ep00.epimg.net/rss/tags/ultimas_noticias.xml,esp,el pais,venezuela,ultimo,Últimas noticias,None,actualidad,3,None,0
1,1,2.00,http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml,esp,el pais,venezuela,más leido,Noticias más vistas,None,actualidad,6,None,0
2,2,3.00,http://ep00.epimg.net/rss/economia/portada.xml,esp,el pais,colombia,economía,Portada de Economía,None,economía,10,None,0
3,3,4.00,https://www.clarin.com/rss/lo-ultimo/,esp,el clarín,argentina,lo-ultimo,"Home, lo último",None,actualidad,6,None,0
4,4,5.00,https://www.clarin.com/rss/economia/,esp,el clarín,argentina,economia,Economía,None,economía,10,None,0
5,5,6.00,https://www.elcomercio.com/feed/,esp,el comercio,perú,None,Lo último,None,actualidad,6,None,0
6,6,7.00,http://www.elcomercio.com/rss/actualidad,esp,el comercio,perú,actualidad,Actualidad,None,actualidad,6,None,1
7,7,8.00,http://cdn.eluniversal.com/rss/avances.xml,esp,el universal,venezuela,avances,"Avances, mala",None,actualidad,6,None,1


In [24]:
#RSS
base_rss=base_rss[base_rss.DescxErrores != 1]
base_rss

#procesamiento de RSS
dicResultado=pd.DataFrame()
bandera=0

for index, row in base_rss.iterrows(): 
    fuenteRss= row["FuentesRSS"]
    idioma=row["lenguaje"]
    pais=row["Pais"]
    
    #depurar antes de procesar
    print ("procesando fuente numero: "+str(index))
    print ( "fuenteRss:"+str(fuenteRss)+" idioma:"+str(idioma)+ " topico:"+str(pais) )   
    
    try:
        auxParse=parse(fuenteRss)
        articles1=get_articles_resultado(auxParse,fuenteRss)
        #articles1    
        #convertir a dataframe
        articles2=pd.DataFrame(articles1)
        #implementacion fechas a articulos
        articles2
        #depurar fecha recibida        
    
        articles2["date"]=articles2["published"].apply(getFechaFromRss)
        articles2["lang"]=idioma
        
        articles2["texto_Limpio"]=articles2["texto"].apply(limpiar_texto)        
        
        if idioma == "eng" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosIngles) 
        elif idioma == "esp" :
            articles2["polaridad"]=articles2["texto_Limpio"].apply(analisisSentimientosEspanol)
        #analisis sentimientos 2
        articles2['polaridad_SQDM']=articles2["texto_Limpio"].apply(sentiment.sentiment)*2-1
        bandera=bandera+1
        if bandera==1:
            dicResultado=articles2
        else:
            dicResultado=dicResultado.append(articles2)
        
    except Exception as e:
        print(e)

procesando fuente numero: 0
fuenteRss:http://ep00.epimg.net/rss/tags/ultimas_noticias.xml idioma:esp topico:venezuela
procesando fuente numero: 1
fuenteRss:http://ep00.epimg.net/rss/tags/noticias_mas_vistas.xml idioma:esp topico:venezuela
procesando fuente numero: 2
fuenteRss:http://ep00.epimg.net/rss/economia/portada.xml idioma:esp topico:colombia
procesando fuente numero: 3
fuenteRss:https://www.clarin.com/rss/lo-ultimo/ idioma:esp topico:argentina
procesando fuente numero: 4
fuenteRss:https://www.clarin.com/rss/economia/ idioma:esp topico:argentina
procesando fuente numero: 5
fuenteRss:https://www.elcomercio.com/feed/ idioma:esp topico:perú
procesando fuente numero: 8
fuenteRss:http://elnacional.com.do/feed/ idioma:esp topico:venezuela
procesando fuente numero: 9
fuenteRss:http://www.eltiempo.com/rss/economia.xml idioma:esp topico:colombia
procesando fuente numero: 10
fuenteRss:http://www.eltiempo.com/rss/deportes.xml idioma:esp topico:colombia
procesando fuente numero: 11
fuenteRss

'published'
procesando fuente numero: 78
fuenteRss:http://rssfeeds.usatoday.com/UsatodaycomNation-TopStories idioma:eng topico:usa
procesando fuente numero: 79
fuenteRss:http://feeds.washingtonpost.com/rss/rss_blogpost idioma:eng topico:usa
procesando fuente numero: 80
fuenteRss:https://www.eltiempo.com/rss/mundo_latinoamerica.xml idioma:esp topico:colombia
procesando fuente numero: 81
fuenteRss:https://www.eltiempo.com/rss/mundo_eeuu-y-canada.xml idioma:esp topico:colombia
procesando fuente numero: 82
fuenteRss:http://www.eltiempo.com/rss/mundo_europa.xml idioma:esp topico:colombia
procesando fuente numero: 83
fuenteRss:http://www.eltiempo.com/rss/mundo_medio-oriente.xml idioma:esp topico:colombia
procesando fuente numero: 84
fuenteRss:http://www.eltiempo.com/rss/mundo_asia.xml idioma:esp topico:colombia
procesando fuente numero: 85
fuenteRss:https://www.eltiempo.com/rss/mundo_africa.xml idioma:esp topico:colombia
procesando fuente numero: 86
fuenteRss:http://www.eltiempo.com/rss/econ

In [25]:
dicResultado['polaridad_SQDM']=dicResultado['polaridad_SQDM'].round(3)
dicResultado['polaridad']=dicResultado['polaridad'].round(3)

In [26]:
dicResultado.sample(10)

,fuenteRss,idP,texto,published,date,lang,texto_Limpio,polaridad,polaridad_SQDM
14,http://www.nytimes.com/services/xml/rss/nyt/SmallBusiness.xml,https://www.nytimes.com/2021/12/20/business/jury-verdict-deliberations-elizabeth-holmes.html,"Jury in Elizabeth Holmes’s Fraud Trial Has Begun Deliberations. The trial, which has stretched nearly four months, has come to represent a critical moment for the tech industry and its culture of overly optimistic salesmanship.","(2021, 12, 20, 22, 56, 32, 0, 354, 0)",2021-12-20 22:56:32.000000,eng,jury in elizabeth holmes fraud trial has begun deliberations the trial which has stretched nearly four months has come to represent critical moment for the tech industry and its culture of overly optimistic salesmanship,-0.59,-0.93
16,http://www.nytimes.com/services/xml/rss/nyt/MediaandAdvertising.xml,https://www.nytimes.com/2022/01/23/us/trevor-jacob-youtube-plane-investigation.html,A YouTuber’s Plane Crash Draws Doubts From Aviation Experts. The video from December shows Trevor Jacob cursing in panic before leaping with a parachute out of a small plane. Some experts wonder if the crash was a publicity stunt.,"(2022, 1, 24, 22, 43, 34, 0, 24, 0)",2022-01-24 22:43:34.000000,eng,a youtuber plane crash draws doubts from aviation experts the video from december shows trevor jacob cursing in panic before leaping with parachute out of small plane some experts wonder if the crash was publicity stunt,-0.87,0.76
8,http://www.portafolio.co/rss/economia/finanzas,https://www.portafolio.co/economia/finanzas/alerta-sobre-pagina-fraudulenta-que-recoge-datos-de-los-ciudadanos-561074,Alerta por página que roba datos de los ciudadanos suplantando la Dian.,"(2022, 1, 28, 18, 58, 56, 4, 28, 0)",2022-01-28 18:58:56.000000,esp,alerta por página que roba datos de los ciudadanos suplantando la dian,-0.98,-0.96
45,http://feeds.nytimes.com/nyt/rss/Business,https://www.nytimes.com/2022/01/27/health/omicron-covid-pandemic.html,"Yes, Omicron Is Loosening Its Hold. But the Pandemic Has Not Ended.. With spotty immunity in the population and a churn of new variants, the coronavirus is likely to become a persistent but hopefully manageable threat.","(2022, 1, 27, 8, 0, 14, 3, 27, 0)",2022-01-27 08:00:14.000000,eng,yes omicron is loosening its hold but the pandemic has not ended with spotty immunity in the population and churn of new variants the coronavirus is likely to become persistent but hopefully manageable threat,-0.05,0.40
1,http://www.eltiempo.com/rss/tecnosfera.xml,https://www.eltiempo.com/tecnosfera/novedades-tecnologia/camaras-de-seguridad-como-prevenir-robo-de-videos-personales-647996,Claves para evitar que 'hackers' ingresen a sus cámaras de seguridad. Cerca de 47 millones de ataques informáticos se presentaron en el primer semestre de 2021.,"(2022, 1, 30, 5, 0, 0, 6, 30, 0)",2022-01-30 05:00:00.000000,esp,claves para evitar que hackers ingresen sus cámaras de seguridad cerca de millones de ataques informáticos se presentaron en el primer semestre de,-0.77,-0.98
7,http://elnacional.com.do/feed/,https://elnacional.com.do/?p=1323892,"Destaca alcances acuerdo con la FAO. Santo Domingo- El ministro de Turismo, David Collado, destacó los alcances del acuerdo firmado entre el Ministerio de Turismo (Mitur) con la Organización de las Naciones Unidas para la Alimentación y Agricultura (FAO), en el que contribuirá a dinamizar la economía con el apoyo a las mipymes rurales aportando más puestos de trabajo en favor [&#8230;]","(2022, 1, 30, 14, 34, 35, 6, 30, 0)",2022-01-30 14:34:35.000000,esp,destaca alcances acuerdo con la fao santo domingo el ministro de turismo david collado destacó los alcances del acuerdo firmado entre el ministerio de turismo mitur con la organización de las naciones unidas para la alimentación agricultura fao en el que contribuirá dinamizar la economía con el apoyo las mipymes rurales aportando más puestos de trabajo en favor,0.04,-0.93
10,http://feeds.washingtonpost.com/rss/sports,https://www.washingtonp

### Guardar en la base de datos

In [36]:

import mysql.connector
import sqlalchemy

database_username = 'root'
database_password = 'Cidifo03%%'
database_ip       = '127.0.0.1:3307'
database_name     = 'tesis'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()


In [30]:
tweetsDFProcesado.drop(columns=['full_text']).to_sql(con=database_connection, name='t_tweets_new_tmp', if_exists='replace',chunksize=100)

In [31]:
dicResultado.drop(columns=['published','texto']).to_sql(con=database_connection, name='t_rss_new_tmp', if_exists='replace',chunksize=100)

## Guardar en tablas principales

In [32]:
import mysql.connector
#host="127.0.0.1:3307",
mydb = mysql.connector.connect(
  host="localhost",
  port=3307,
  user="root",
  password="Cidifo03%%",
  database="tesis"
)

In [33]:
mycursor = mydb.cursor()

In [34]:
sql_act_tweets = "delete t1 from tesis.t_tweets_new t1 inner join tesis.t_tweets_new_tmp t2 on (t1.id=t2.id and t1.created_at=t2.created_at)"
sql_act_rss = "delete t1 from tesis.t_rss_new t1 inner join tesis.t_rss_new_tmp t2 on (t1.fuenteRss=t2.fuenteRss and t1.idP=t2.idP and t1.date=t2.date)"

In [35]:
mycursor.execute(sql_act_tweets)
mydb.commit()

In [36]:
mycursor.execute(sql_act_rss)
mydb.commit()

In [37]:
tweetsDFProcesado.drop(columns=['full_text']).to_sql(con=database_connection, name='t_tweets_new', if_exists='append',chunksize=100)
dicResultado.drop(columns=['published','texto']).to_sql(con=database_connection, name='t_rss_new', if_exists='append',chunksize=100)

In [38]:
### Cerrar nuevamente al final del script
connection.close()
database_connection.close()
mydb.close()

### Próximos pasos

* Definir si se va aguardar información de tokenización
* Definir si se van a definir tópicos generales o por fuente para segmentar los textos y buscar correlaciones.
* try cath RSS
* validar cierre conexión

# Leer y guardar variables dependientes (TRM y precios)

In [104]:
from datetime import datetime, timedelta

In [105]:
date=time.strftime('%Y-%m-%d')

In [121]:
d_start = datetime.today() - timedelta(days=15)
d_start=d_start.strftime('%Y-%m-%d')
d_start

'2022-01-30'

###  TRM


In [122]:
#!/usr/bin/env python
from suds.client import Client
import time

WSDL_URL = 'https://www.superfinanciera.gov.co/SuperfinancieraWebServiceTRM/TCRMServicesWebService/TCRMServicesWebService?WSDL'
#date = time.strftime('%Y-%m-%d')

def trm(date):
    try:
        client = Client(WSDL_URL, location=WSDL_URL, faults=True)
        trm =  client.service.queryTCRM(date)
    except Exception as e:
        return str(e)

    return trm

print (trm(date))
resultado_trm=trm(date)

(tcrmResponse){
   id = 1376001
   unit = "COP"
   validityFrom = 2022-02-12 00:00:00-05:00
   validityTo = 2022-02-14 00:00:00-05:00
   value = 3917.52
   success = True
 }


In [123]:
resultado_trm

In [124]:
## Crear diccionario para guardar resultado
resultado_trm_dic={}

In [125]:
for i,j in resultado_trm:
    print(i,j)
    resultado_trm_dic[i]=j

id 1376001
unit COP
validityFrom 2022-02-12 00:00:00-05:00
validityTo 2022-02-14 00:00:00-05:00
value 3917.52
success True


In [126]:
resultado_trm_dic

{'id': 1376001,
 'unit': 'COP',
 'validityFrom': datetime.datetime(2022, 2, 12, 0, 0, tzinfo=<suds.sax.date.FixedOffsetTimezone object at 0x0000019E1FBAA160>),
 'validityTo': datetime.datetime(2022, 2, 14, 0, 0, tzinfo=<suds.sax.date.FixedOffsetTimezone object at 0x0000019E1FBAA1C0>),
 'value': 3917.52,
 'success': True}

In [127]:
pd.DataFrame.from_dict(resultado_trm_dic, orient='index')

,0
id,1376001
unit,COP
validityFrom,2022-02-12 00:00:00-05:00
validityTo,2022-02-14 00:00:00-05:00
value,3917.52
success,True


In [128]:
resultado_trm_dic=pd.DataFrame.from_dict(resultado_trm_dic, orient='index').T
resultado_trm_dic

,id,unit,validityFrom,validityTo,value,success
0,1376001,COP,2022-02-12 00:00:00-05:00,2022-02-14 00:00:00-05:00,3917.52,True


In [129]:
date

'2022-02-14'

In [130]:
try:
    resultado_trm_dic.to_sql(con=database_connection, name='t_trm_new', if_exists='append',chunksize=100)
except:
    print('ya habia sido guardada')

ya habia sido guardada


## Precios
https://ichi.pro/es/predecir-los-precios-del-petroleo-con-aprendizaje-automatico-y-python-55256159639482
https://ichi.pro/es/python-he-probado-la-api-de-quandl-y-como-obtener-datos-economicos-y-inmobiliarios-en-una-linea-de-codigo-11078875261371

In [131]:
import quandl

In [132]:
# Setting our API key
quandl.ApiConfig.api_key = 'X6mZSY79yeySfpCVJaKg'

# Importing our data
data_petroleo = quandl.get("FRED/DCOILBRENTEU", start_date=d_start, end_date=date)

In [133]:
data_petroleo.tail(10)

,Value
Date,
2022-01-31,92.35
2022-02-01,90.24
2022-02-02,91.43
2022-02-03,92.99
2022-02-04,96.86
2022-02-07,97.28


In [134]:
date

'2022-02-14'

In [135]:
data_oro = quandl.get("LBMA/GOLD", start_date=d_start, end_date=date)

In [136]:
data_oro[['USD (PM)']]

,USD (PM)
Date,
2022-01-31,1795.25
2022-02-01,1799.85
2022-02-02,1803.65
2022-02-03,1792.70
2022-02-04,1804.70
2022-02-07,1813.55
2022-02-08,1822.60
2022-02-09,1827.80
2022-02-10,1835.35


In [139]:
df_precios=data_petroleo.merge(data_oro[['USD (PM)']],left_index=True,right_index=True,how='outer')

In [141]:
df_precios.columns=['petroleo','oro']

In [144]:
df_precios.to_sql(con=database_connection, name='t_precios_tmp', if_exists='replace')

In [ ]:
sql_act_precios = "delete t1 from tesis.t_precios t1 inner join tesis.t_precios_tmp t2 on (t1.Date=t2.Date)"

In [ ]:
mycursor.execute(sql_act_precios)
mydb.commit()

In [ ]:
df_precios.to_sql(con=database_connection, name='t_precios_tmp', if_exists='append')